In [1]:
import numpy as np
import datetime
import spiceypy

# Load the SPICE kernel
spiceypy.furnsh('kernels/spk/de432s.bsp')
spiceypy.furnsh('kernels/spk/codes_300ast_20100725.bsp')
spiceypy.furnsh('kernels/misc/codes_300ast_20100725.tf')
spiceypy.furnsh('kernels/lsk/naif0012.tls')
spiceypy.furnsh('kernels/pck/gm_de431.tpc')

# Define the UTC and ET
utc = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
et = spiceypy.utc2et(utc)

In [2]:
# sxform transforms the matrix of the instring frame to the matrix of the tostring frame at the epoch et
matrix = spiceypy.sxform(instring="ECLIPJ2000_DE405", tostring="ECLIPJ2000", et=et)

# get the state vector of ceres
ceres_state_vector, _ = spiceypy.spkgeo(targ=2000001, et=et, ref='ECLIPJ2000', obs=10)

# gravitational parameter of the sun
_, gravitational_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)

# get the float
gravitational_sun = gravitational_sun_pre[0]

# oscltx takes the state vector, corresponding ET and the gravitational parameter to calculate the orbital elements for that particular time
ceres_orbital_elements = spiceypy.oscltx(state=ceres_state_vector, et=et, mu=gravitational_sun)

ceres_semi_major_astronomical_units = spiceypy.convrt(ceres_orbital_elements[9], inunit='km', outunit='AU')
ceres_perihelion_astronomical_units = spiceypy.convrt(ceres_orbital_elements[0], inunit='km', outunit='AU')
eccentricity_ceres = np.degrees(ceres_orbital_elements[1])
inclination_ceres_degrees = np.degrees(ceres_orbital_elements[2])
longitude_node_ceres_degrees = np.degrees(ceres_orbital_elements[3])
argument_perihelion_ceres_degrees = np.degrees(ceres_orbital_elements[4])

ceres_orbit_time_years = ceres_orbital_elements[10] / (365.0 * 86400.0)

In [3]:
# conics takes the orbital elements and the ET to calculate the state vector of the body
ceres_state_re = spiceypy.conics([ceres_orbital_elements[0], ceres_orbital_elements[1], 
                                    ceres_orbital_elements[2], ceres_orbital_elements[3], ceres_orbital_elements[4],
                                     ceres_orbital_elements[5], ceres_orbital_elements[6], gravitational_sun], et)

print(ceres_state_re)
print(ceres_state_vector)

[ 1.18420804e+00 -1.37469843e-01  2.66616786e+00  1.42592677e+05
 -4.74691334e+05  1.70038695e+05]
[-5.55369215e+07 -4.18470124e+08 -3.01065043e+06  1.68819012e+01
 -3.57220944e+00 -3.22305659e+00]
